In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [2]:
df = pd.read_csv('/Users/priya/Downloads/train.csv')

In [ ]:
print(df['TARGET_5Yrs'].value_counts())

In [3]:
target = df.pop('TARGET_5Yrs')

In [4]:
df.drop('Id_old', axis=1, inplace=True)
df.drop('Id', axis=1, inplace=True)

In [5]:
df.columns = df.columns.str.strip()
df

GP   MIN   PTS  FGM  FGA   FG%  3P Made  3PA   3P%  FTM  FTA   FT%  \
0     80  24.3   7.8  3.0  6.4  45.7      0.1  0.3  22.6  2.0  2.9  72.1   
1     75  21.8  10.5  4.2  7.9  55.1     -0.3 -1.0  34.9  2.4  3.6  67.8   
2     85  19.1   4.5  1.9  4.5  42.8      0.4  1.2  34.3  0.4  0.6  75.7   
3     63  19.1   8.2  3.5  6.7  52.5      0.3  0.8  23.7  0.9  1.5  66.9   
4     63  17.8   3.7  1.7  3.4  50.8      0.5  1.4  13.7  0.2  0.5  54.0   
...   ..   ...   ...  ...  ...   ...      ...  ...   ...  ...  ...   ...   
7995  32   9.2   1.8  0.7  1.8  40.3     -0.1 -0.2  23.1  0.4  0.6  65.7   
7996  54   6.0   1.8  0.7  1.4  48.7      0.1  0.1   3.1  0.2  0.4  70.1   
7997  85  28.2  10.7  4.0  9.0  45.1      0.2  0.6  23.6  2.8  3.9  69.7   
7998  39   7.7   2.5  1.0  2.3  40.1     -0.3 -0.5  13.3  0.6  0.7  74.3   
7999  49  19.2   4.8  1.7  5.1  32.6      0.7  2.4  41.3  0.8  1.3  70.1   

      OREB  DREB  REB  AST  STL  BLK  TOV  
0      2.2   2.0  3.8  3.2  1.1  0.2  1.6  
1      3.6   3.7  6.6  0.7  0.5  0.6  1.4  
2      0.6   1.8  2.4  0.8  0.4  0.2  0.6  
3      0.8   2.0  3.0  1.8  0.4  0.1  1.9  
4      2.4   2.7  4.9  0.4  0.4  0.6  0.7  
...    ...   ...  ...  ...  ...  ...  ...  
7995   0.3   1.8  1.9  0.5  0.3  0.2  0.4  
7996   1.0   1.1  2.0  0.1  0.0  0.3  0.3  
7997   1.0   2.1  3.1  3.4  1.2  0.2  1.8  
7998   0.4   0.6  0.9  0.2  0.3  0.3  0.5  
7999   0.1   1.1  1.2  3.5  0.9 -0.3  1.4  

[8000 rows x 19 columns]

In [6]:
#df_cleaned = df.drop(['FG%', '3P%', 'FT%'], axis = 1) 

In [ ]:
df.isnull().sum()

duplicate = df.duplicated()
print(duplicate.sum())

In [7]:
from imblearn.over_sampling import SMOTE 

In [8]:
scaler = StandardScaler()
df_cleaned = scaler.fit_transform(df)

In [9]:
sm = SMOTE(random_state=42)

X_sm, y_sm = sm.fit_resample(df_cleaned, target)

print(f'''Shape of X before SMOTE: {df.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print('\nBalance of positive and negative classes (%):')
y_sm.value_counts(normalize=True) * 100

Shape of X before SMOTE: (8000, 19)
Shape of X after SMOTE: (13338, 19)

Balance of positive and negative classes (%):


1    50.0
0    50.0
Name: TARGET_5Yrs, dtype: float64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=8)

In [11]:
#X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=8)

In [12]:
X_test

array([[ 0.37633754, -0.87949075, -0.93669839, ..., -1.24637649,
        -0.00554072, -1.26806809],
       [-0.0945194 , -1.03111353, -1.03256468, ..., -1.10080177,
        -0.14765733, -1.0477545 ],
       [ 1.59029069,  1.04349704,  1.53594318, ...,  1.84325645,
         1.04117255,  1.71763572],
       ...,
       [-1.57559601,  0.11709992,  0.33856468, ..., -0.53090186,
         0.02740114,  0.66045036],
       [-0.22415727, -0.59866487, -0.58527364, ..., -0.66112901,
         0.01849247, -1.10524449],
       [-0.22069987, -1.20615771, -1.196511  , ..., -0.85546358,
        -0.17687585, -0.63294597]])

In [13]:
from sklearn import svm

clf = svm.SVC(kernel='linear', probability = True) # Linear Kernel
#clf = svm.SVC(C = 10, gamma = 0.01, kernel = 'linear', probability = True)
#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [14]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6619190404797601


In [15]:
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 0.68578352180937
Recall: 0.6233480176211453


In [16]:
clf.score(X_test, y_test)
clf.predict_proba(X_test)

array([[0.48785316, 0.51214684],
       [0.48277409, 0.51722591],
       [0.15935058, 0.84064942],
       ...,
       [0.70698182, 0.29301818],
       [0.5199803 , 0.4800197 ],
       [0.62588756, 0.37411244]])

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[917 389]
 [513 849]]


In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.70      0.67      1306
           1       0.69      0.62      0.65      1362

    accuracy                           0.66      2668
   macro avg       0.66      0.66      0.66      2668
weighted avg       0.66      0.66      0.66      2668



In [19]:
roc=roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
roc

0.731666565473259

In [20]:
#from sklearn.model_selection import GridSearchCV 
#from sklearn.svm import SVC 
  
# defining parameter range 
#param_grid = {'C': [0.1, 1, 10, 100, 1000],  
 #             'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
  #            'kernel': ['rbf']}  
  
#grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
#grid.fit(X_train, y_train) 

In [21]:
# print best parameter after tuning 
#print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 

In [22]:
#grid_predictions = grid.predict(X_test) 
  
# print classification report 
#print(classification_report(y_test, grid_predictions)) 

In [23]:
#roc=roc_auc_score(y_test, xgb_cv.predict_proba(X_test)[:,1])
#roc


In [24]:
df1 = pd.read_csv('/Users/priya/Downloads/test.csv')
df1.drop('Id', axis=1, inplace=True)
df1.drop('Id_old', axis=1, inplace=True)

In [25]:
#df2 = df1.drop(['FG%', '3P%', 'FT%'], axis = 1) 
df1_cleaned = scaler.fit_transform(df1)

In [26]:
final = clf.predict_proba(df1_cleaned)[:,1]

In [27]:
final

array([0.5       , 0.46443241, 0.68956639, ..., 0.31197637, 0.81367904,
       0.37997486])

In [28]:
pd.DataFrame(final).to_csv("/Users/priya/Downloads/13684182_week2_svm_smote.csv")